# Do inference sequentially

In [ ]:
import os
import glob
import pickle
import time
import requests

In [ ]:
from get_inference_service_url import get_inference_service_url
infer_url = get_inference_service_url()

In [ ]:
# how many requests to send
MAX_REQUESTS=50

In [ ]:
# Function to send a single REST request
def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "pixel_values",
                "shape": data.shape,
                "datatype": "FP32",
                "data": data.tolist()
            }
        ]
    }

    start_time = time.time()
    response = requests.post(infer_url, json=json_data, verify=True)
    end_time = time.time()

    if response.status_code != 200:
        print(f"Request failed with status code: {response.status_code}")


    response_time = end_time - start_time
    print(f"Response time: {response_time:.6f} seconds")

    return response

In [ ]:
pickled_file = "pickled/pickled_tensors.pkl"

with open(pickled_file, 'rb') as file:
    mytensors = pickle.load(file)

# Limit the output to the first 50 .pkl files
limited_tensors = list(mytensors.values())[:MAX_REQUESTS]

start_time = time.time()  # record start time
request_count = 0  # initialize request counter

for tensor in limited_tensors:
    pixel_values = tensor["pixel_values"]
    rest_request(pixel_values)
    request_count += 1  # increment request counter

end_time = time.time()  # record end time

elapsed_time = end_time - start_time  # calculate elapsed time
average_time = elapsed_time / request_count  # compute average time per request

print(f"Total requests processed: {request_count}")
print(f"Time taken: {elapsed_time} seconds")
print(f"Average time per request: {average_time} seconds")